In [1]:
!nvidia-smi

Sat Sep 14 16:39:11 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.06             Driver Version: 535.183.06   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       On  | 00000000:00:1E.0 Off |                    0 |
| N/A   31C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import os

In [3]:
# Space I have 
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay         100G   30G   71G  30% /
tmpfs            64M     0   64M   0% /dev
tmpfs           7.7G     0  7.7G   0% /sys/fs/cgroup
/dev/nvme0n1p1  100G   30G   71G  30% /run
tmpfs            14G     0   14G   0% /dev/shm
/dev/nvme2n1    2.0G  3.3M  1.9G   1% /home/jovyan
tmpfs            14G  120K   14G   1% /home/jovyan/.saturn
tmpfs            14G   12K   14G   1% /run/secrets/kubernetes.io/serviceaccount
tmpfs           7.7G   12K  7.7G   1% /proc/driver/nvidia
tmpfs           7.7G  3.0M  7.7G   1% /run/nvidia-persistenced/socket
tmpfs           7.7G     0  7.7G   0% /proc/acpi
tmpfs           7.7G     0  7.7G   0% /sys/firmware


In [4]:
# Checking all the environment variables available
os.environ

environ{'SHELL': '/bin/bash',
        'NV_LIBCUBLAS_VERSION': '11.11.3.6-1',
        'NVIDIA_VISIBLE_DEVICES': 'all',
        'KUBERNETES_SERVICE_PORT_HTTPS': '443',
        'KUBERNETES_SERVICE_PORT': '443',
        'SATURN_USER': 'khandelwalvaibhav23',
        'CONDA_BIN': '/opt/saturncloud/bin',
        'BASE_URL': 'https://app.community.saturnenterprise.io',
        'HOSTNAME': 'w-khand-llmgpu-cd5aa824568e4153ac6486d037e258da-59b56667457m9f5',
        'SATURN_TOKEN': 'server-1e1d1d5d02384e1cb06ae5b0afb35eb7',
        'LANGUAGE': 'en_US.UTF-8',
        'SATURN_RESOURCE_TYPE': 'SingleUserServer',
        'SATURN_EMAIL': 'khandelwalvaibhav23@gmail.com',
        'SATURN_SSH_ENABLED': 'false',
        'SATURN_RESOURCE_NAME': 'LLM_GPU',
        'NVIDIA_REQUIRE_CUDA': 'cuda>=11.8 brand=tesla,driver>=450,driver<451 brand=tesla,driver>=470,driver<471 brand=unknown,driver>=470,driver<471 brand=nvidia,driver>=470,driver<471 brand=nvidiartx,driver>=470,driver<471 brand=geforce,driver>=470,drive

In [6]:
# As hugging face stores the file inside the cache folder inside HOME directory which in our case doesn't have mmuch storage, so changing the default storage of Hugging Face
# HF_HOME is the variable which contains the location to store the Hugging Face data
# MAKE SURE TO RESTART THE KERNEL AFTER THIS
os.environ['HF_HOME'] = '/run/cache/'

In [7]:
os.environ['HF_HOME']

'/run/cache/'

# Copied from Hugging Face <a href="https://huggingface.co/google/flan-t5-xl"> Webage of the model</a> 

In [8]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

[2024-09-14 16:41:59,647] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [9]:
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-xl")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-xl", device_map="auto")

You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [23]:
input_text = "Write in a sentence, what is apple?"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")
input_ids

tensor([[8733,   16,    3,    9, 7142,    6,  125,   19, 8947,   58,    1]],
       device='cuda:0')

In [24]:
outputs = model.generate(input_ids)
outputs

tensor([[   0, 2184,   19,    3,    9, 2728,    5,    1]], device='cuda:0')

In [25]:
print(tokenizer.decode(outputs[0]))

<pad> Apple is a fruit.</s>


From above it can be seen that <pad> and </s> is denoting the starting and end of the output having 0 and 1 as the output value

# <span style="color:blue">Combining everything together to form a RAG

In [27]:
# Getting the Similar Doc Search python file created in Part_01_Basics
!rm -f similarDocSearch.py
!wget https://raw.githubusercontent.com/vaibhavcodes/LLM_RAG/main/Part_01_Basics/similarDocSearch.py

--2024-09-14 17:15:53--  https://raw.githubusercontent.com/vaibhavcodes/LLM_RAG/main/Part_01_Basics/similarDocSearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3320 (3.2K) [text/plain]
Saving to: ‘similarDocSearch.py’

similarDocSearch.py 100%[===================>]   3.24K  --.-KB/s    in 0s      

2024-09-14 17:15:53 (78.9 MB/s) - ‘similarDocSearch.py’ saved [3320/3320]



In [29]:
from similarDocSearch import *
import json
import requests

In [44]:
# Documents fetching from the github
docs_url = "https://github.com/vaibhavcodes/LLM_RAG/blob/main/Part_01_Basics/data/documents.json?raw=1"
docs_response = requests.get(docs_url)
docs_raw = docs_response.json()
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course'] 
        documents.append(doc)

In [42]:
# Initializing the class similarDocSearch
obj = similarDocSearch(text_fields = ["question", "text", "section"])
# Fitting
obj.fit(documents)

In [69]:
# Function that would search the most similar documents to our query and will return it
def search(query):
    boost = {'question': 3, 
         'section': 0.5} # Used to give a field more or less importance 
    
    filter_course = "data-engineering-zoomcamp"
    num_results = 5
    
    results = obj.search(query = query, boosts=boost, filter_course=filter_course, num_results=num_results)
    return results

In [70]:
# Function that would build up the prompt
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
    Use only the facts from the CONTEXT when answering the QUESTION.
    If the CONTEXT doesn't contains the answer, output None
    
    QUESTION: {question}
    
    CONTEXT: 
    {context}
    """.strip()

    # Context creation to be passed into the prompt
    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\n question: {doc['question']}\n answer: {doc['text']}\n \n"
    
    # Prompt formatting
    prompt = prompt_template.format(question=query, context=context).strip()
    
    return(prompt)

In [74]:
# Function that would use LLM
def llm(prompt):
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")
    outputs = model.generate(input_ids, max_length=100)
    return (tokenizer.decode(outputs[0]))

In [75]:
# Function that would make a whole RAG
def RAG(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [76]:
q = "I just discovered the course. Can i still join it??"
print(RAG(q))

<pad> Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.</s>
